## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_clean = application_df.drop(columns=["EIN", "NAME"])

In [ ]:
# Determine the number of unique values in each column.
for column in application_df_clean.columns:
    print(f"{column} has {application_df_clean[column].nunique()} unique values")

APPLICATION_TYPE has 17 unique values
AFFILIATION has 6 unique values
CLASSIFICATION has 71 unique values
USE_CASE has 5 unique values
ORGANIZATION has 4 unique values
STATUS has 2 unique values
INCOME_AMT has 9 unique values
SPECIAL_CONSIDERATIONS has 2 unique values
ASK_AMT has 8747 unique values
IS_SUCCESSFUL has 2 unique values


In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df_clean["APPLICATION_TYPE"].value_counts()
print(application_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
application_types_to_replace = application_counts[application_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df_clean["CLASSIFICATION"].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_numeric = pd.get_dummies(application_df_clean, columns=["APPLICATION_TYPE",
                                                                       "AFFILIATION",
                                                                       "CLASSIFICATION",
                                                                       "USE_CASE",
                                                                       "ORGANIZATION",
                                                                       "INCOME_AMT",
                                                                       "SPECIAL_CONSIDERATIONS"])

In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df_numeric.drop(columns=["IS_SUCCESSFUL"])
y = application_df_numeric["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[1]))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                8720      
                                                                 
 dense_13 (Dense)            (None, 80)                6480      
                                                                 
 dense_14 (Dense)            (None, 80)                6480      
                                                                 
 dense_15 (Dense)            (None, 1)                 81        
                                                                 
Total params: 21761 (85.00 KB)
Trainable params: 21761 (85.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
train_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5726 - accuracy: 0.7213
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5513 - accuracy: 0.7321
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7331
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7338
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7339
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5437 - accuracy: 0.7358
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accuracy: 0.7359
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5423 - accuracy: 0.7371
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5417 - accuracy: 0.7370
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5414 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5926 - accuracy: 0.7278 - 683ms/epoch - 3ms/step
Loss: 0.5925647616386414, Accuracy: 0.7278134226799011


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Adjust the input data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Bin the NAME column where names occurring less than 5 times are labeled as "Other"
name_counts = application_df['NAME'].value_counts()
names_to_replace = name_counts[name_counts < 5].index
application_df['NAME'] = application_df['NAME'].replace(names_to_replace, "Other")
application_df = application_df.drop(columns=["EIN"])


In [ ]:
# Adjust bins
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_counts[application_counts < 500].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < 1000].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other")


In [ ]:
application_df = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS", "NAME"])


In [ ]:
# Rebuild the model with adjustments
import tensorflow as tf

# Define the model and add more hidden layers
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=len(application_df.columns)-1))
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dropout(0.2))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Retrain with more epochs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = nn.fit(X_train_scaled, y_train, epochs=150, callbacks=[early_stopping], validation_split=0.2, verbose=2)


Epoch 1/150
644/644 - 2s - loss: 0.4118 - accuracy: 0.8064 - val_loss: 0.4586 - val_accuracy: 0.7901 - 2s/epoch - 2ms/step
Epoch 2/150
644/644 - 2s - loss: 0.4111 - accuracy: 0.8066 - val_loss: 0.4586 - val_accuracy: 0.7946 - 2s/epoch - 3ms/step
Epoch 3/150
644/644 - 2s - loss: 0.4108 - accuracy: 0.8056 - val_loss: 0.4573 - val_accuracy: 0.7930 - 2s/epoch - 3ms/step
Epoch 4/150
644/644 - 2s - loss: 0.4088 - accuracy: 0.8072 - val_loss: 0.4610 - val_accuracy: 0.7936 - 2s/epoch - 3ms/step
Epoch 5/150
644/644 - 2s - loss: 0.4099 - accuracy: 0.8064 - val_loss: 0.4666 - val_accuracy: 0.7918 - 2s/epoch - 2ms/step
Epoch 6/150
644/644 - 2s - loss: 0.4085 - accuracy: 0.8066 - val_loss: 0.4589 - val_accuracy: 0.7918 - 2s/epoch - 3ms/step
Epoch 7/150
644/644 - 2s - loss: 0.4074 - accuracy: 0.8070 - val_loss: 0.4736 - val_accuracy: 0.7913 - 2s/epoch - 4ms/step
Epoch 8/150
644/644 - 2s - loss: 0.4080 - accuracy: 0.8066 - val_loss: 0.4590 - val_accuracy: 0.7924 - 2s/epoch - 3ms/step
Epoch 9/150
644/

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4610 - accuracy: 0.7883 - 345ms/epoch - 1ms/step
Loss: 0.46104004979133606, Accuracy: 0.7883381843566895


In [ ]:
nn.save("AlphabetSoupCharity_Optimization.h5")

**Overview**

This analysis focused on using a deep learning model to predict the outcome of funding applications submitted to Alphabet Soup, a philanthropic organization. The objective was to discern patterns that influence the likelihood of an application's success.

**Results**

*Data Preprocessing*

Target Variable: The model's target was IS_SUCCESSFUL, indicating whether the funding application was approved.

Feature Variables: Excluding identifiers (EIN, NAME) and the target, all other variables were used as features.

Variables Removed: EIN and NAME were removed from the input data, as they do not affect the outcome prediction.

*Model Development*

Model Configuration: The neural network included three hidden layers with 100, 80, and 50 neurons, using ReLU for activation. A sigmoid function was employed in the output layer for binary outcome prediction.

Performance: The model achieved an accuracy of 78%, surpassing the target performance of 75%.

Optimization Steps: To enhance model performance, I adjusted the preprocessing approach, particularly by binning less frequent NAME entries, and tweaked the model by increasing the number of neurons and layers, adding dropout layers, and fine-tuning the training process with more epochs and early stopping.

**Summary**

The deep learning model demonstrated promising results, achieving an accuracy of 78% and exceeding the set goal. This success underscores the model's capability to effectively predict funding application outcomes for Alphabet Soup.

**Recommendation**

Considering your coursework on supervised, unsupervised learning, and neural networks, it's recommended to explore other supervised learning models such as Random Forests or Support Vector Machines (SVMs) for this classification task. These models could offer simpler alternatives with potentially strong performance. Additionally, applying unsupervised techniques like Principal Component Analysis (PCA) for dimensionality reduction could help in refining the feature set, possibly enhancing model accuracy. This combined approach allows for a broad application of your learned concepts while aiming to improve predictions on funding application success.